In [1]:
%matplotlib inline
import axs
import os
from glob import glob

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from astropy.io import ascii

import pandas as pd

from multiprocessing import Pool

import h5py
import pyspark.sql.functions as sparkfunc

In [2]:
from pyspark.sql import SparkSession
spark_session = (SparkSession.builder
         .config('spark.master', "local[10]") # 20 when running the full crossmatch, 10 when doing validation.
         .config('spark.driver.memory', '80G')
         .config("spark.sql.execution.arrow.enabled", "true")
         .config("spark.ui.proxyBase", "https://epyc.astro.washington.edu/jupyter/user/ecbellm/proxy/4040")
         .config("spark.port.maxRetries", "128")
         .enableHiveSupport()
         .getOrCreate())
spark_session

In [3]:
catalog = axs.AxsCatalog(spark_session)

In [4]:
catalog.list_tables().keys()

dict_keys(['gaia_dr2_1am_dup', 'allwise_1am_dup', 'unwise_v1', 'cklein_flare', 'unwise_v2', 'catalina_variables_n', 'gaia', 'gaia_dr2_1am_dup_ssd', 'ps1', 'green19_stellar_params', 'rosat_2rxs', 'rosat_2rxs_z4am_b2am', 'ztf_dr3_2rxs_obj', 'wtf_ztf_dr3', 'ztf_wds_kjb_dr3', 'karenws_cut_wtf_fits_r_band', 'karenws_cut_wtf_fits', 'ztf_aug2020_2rxs_obj', 'skymapper_dr2', 'test_skymapper', 'skymapper_dr2_ver2', 'skymapper_dr2_ver3', 'ztf_rrlyr', 'gaia_source_edr3', 'gaia_edr3_distances', 'rrlyrae_sample_andy', 'stevengs_test_small_df', 'ztf5', 'feh_rrlyr_ab_020620', 'kepler_rrlyrae', 'ztf_kepler_rrlyrae', 'ps_uband', 'ps_uband_ver2', 'debug_match_a', 'debug_match_b', 'debug_match_c', 'ztf_rrlyr_grid_50', 'igaps_halpha_emitters', 'wtf_aug2020_asymmetric_2', 'wtf_aug2020_asymmetric_3', 'wtf_aug2020_dip_candidates', 'ztf_rr_lyrae_preprocessed_and_gridded_09_01_2021', 'ztf_rr_lyrae_phi_31_fits_09_01_2021', 'stevengs_test_sources', 'ddf_sources_bigger', 'ztf_dr7', 'jrad_zg98m', 'jrad_zgd98m', 'jr

## VariSummary table

In [5]:
df_test = spark_session.read.csv('/epyc/data/gaia_dr3/csv/VariSummary/VariSummary_462888-462913.csv.gz',comment='#',header=True,inferSchema=True)

In [6]:
df_test.schema

StructType(List(StructField(solution_id,LongType,true),StructField(source_id,LongType,true),StructField(num_selected_g_fov,IntegerType,true),StructField(mean_obs_time_g_fov,DoubleType,true),StructField(time_duration_g_fov,DoubleType,true),StructField(min_mag_g_fov,DoubleType,true),StructField(max_mag_g_fov,DoubleType,true),StructField(mean_mag_g_fov,DoubleType,true),StructField(median_mag_g_fov,DoubleType,true),StructField(range_mag_g_fov,DoubleType,true),StructField(trimmed_range_mag_g_fov,DoubleType,true),StructField(std_dev_mag_g_fov,DoubleType,true),StructField(skewness_mag_g_fov,DoubleType,true),StructField(kurtosis_mag_g_fov,DoubleType,true),StructField(mad_mag_g_fov,DoubleType,true),StructField(abbe_mag_g_fov,DoubleType,true),StructField(iqr_mag_g_fov,DoubleType,true),StructField(stetson_mag_g_fov,DoubleType,true),StructField(std_dev_over_rms_err_mag_g_fov,DoubleType,true),StructField(outlier_median_g_fov,DoubleType,true),StructField(num_selected_bp,IntegerType,true),StructField

In [8]:
df_test.count()

1065

In [9]:
df_test.head()

Row(solution_id=375316653866487564, source_id=4071605906863950848, num_selected_g_fov=24, mean_obs_time_g_fov=2233.0470813836528, time_duration_g_fov=935.6858, min_mag_g_fov=16.434698, max_mag_g_fov=17.187775, mean_mag_g_fov=16.742523, median_mag_g_fov=16.706411, range_mag_g_fov=0.7530761, trimmed_range_mag_g_fov=0.680446, std_dev_mag_g_fov=0.21599738, skewness_mag_g_fov=0.41057605, kurtosis_mag_g_fov=-0.68246186, mad_mag_g_fov=0.2320748, abbe_mag_g_fov=1.0968395, iqr_mag_g_fov=0.32099915, stetson_mag_g_fov=24.84971, std_dev_over_rms_err_mag_g_fov=45.734005, outlier_median_g_fov=135.3269, num_selected_bp=27, mean_obs_time_bp=2225.785199391947, time_duration_bp=969.8911, min_mag_bp=16.53926, max_mag_bp=17.648405, mean_mag_bp=17.252209, median_mag_bp=17.26781, range_mag_bp=1.1091436, trimmed_range_mag_bp=0.71454364, std_dev_mag_bp=0.25204545, skewness_mag_bp=-0.6977858, kurtosis_mag_bp=0.95739657, mad_mag_bp=0.23522338, abbe_mag_bp=0.8895337, iqr_mag_bp=0.3551742, stetson_mag_bp=5.56147,

In [10]:
df = spark_session.read.csv('/epyc/data/gaia_dr3/csv/VariSummary',comment='#',header=True,schema=df_test.schema)

In [ ]:
# to load into AXS we need ra and dec, so join to our export from GaiaSource

In [11]:
df_radec = spark_session.read.parquet('/epyc/data/gaia_dr3/source_id_ra_dec.parquet')

In [12]:
df_radec.head()

Row(source_id=4116923378115693568, ra=265.4280847078738, dec=-22.989304805079296)

https://stackoverflow.com/questions/33778664/spark-dataframe-distinguish-columns-with-duplicated-name

In [22]:
df_radec_alias = df_radec.select(sparkfunc.col('source_id').alias('source_id_radec'),'ra','dec')


In [23]:
df_radec_alias.head()

Row(source_id_radec=4116923378115693568, ra=265.4280847078738, dec=-22.989304805079296)

In [25]:
# need the select to avoid duplicating the source_id column
(df_radec_alias.join(df_test,df_test.source_id == df_radec_alias.source_id_radec, 'inner')\
    .select('ra','dec',*(sparkfunc.col(x) for x in df_test.columns)).head())

Row(ra=280.20148696019265, dec=-28.96359515484567, solution_id=375316653866487564, source_id=4071605906863950848, num_selected_g_fov=24, mean_obs_time_g_fov=2233.0470813836528, time_duration_g_fov=935.6858, min_mag_g_fov=16.434698, max_mag_g_fov=17.187775, mean_mag_g_fov=16.742523, median_mag_g_fov=16.706411, range_mag_g_fov=0.7530761, trimmed_range_mag_g_fov=0.680446, std_dev_mag_g_fov=0.21599738, skewness_mag_g_fov=0.41057605, kurtosis_mag_g_fov=-0.68246186, mad_mag_g_fov=0.2320748, abbe_mag_g_fov=1.0968395, iqr_mag_g_fov=0.32099915, stetson_mag_g_fov=24.84971, std_dev_over_rms_err_mag_g_fov=45.734005, outlier_median_g_fov=135.3269, num_selected_bp=27, mean_obs_time_bp=2225.785199391947, time_duration_bp=969.8911, min_mag_bp=16.53926, max_mag_bp=17.648405, mean_mag_bp=17.252209, median_mag_bp=17.26781, range_mag_bp=1.1091436, trimmed_range_mag_bp=0.71454364, std_dev_mag_bp=0.25204545, skewness_mag_bp=-0.6977858, kurtosis_mag_bp=0.95739657, mad_mag_bp=0.23522338, abbe_mag_bp=0.8895337

In [26]:
%%time

df_joined = (df_radec_alias.join(df,df.source_id == df_radec_alias.source_id_radec, 'inner')\
    .select('ra','dec',*(sparkfunc.col(x) for x in df.columns)))

catalog.save_axs_table(df_joined, "gaia_dr3_vari_summary", calculate_zone=True)

CPU times: user 202 ms, sys: 78.8 ms, total: 280 ms
Wall time: 9min 8s


## test Gaia Source

In [5]:
gaia = catalog.load("gaia_dr3_source")

In [27]:
vari = catalog.load("gaia_dr3_vari_summary")

In [29]:
vari.count()

13715388

In [30]:
vari.head()

Row(ra=2.0565882573506062, dec=-84.93676073039339, solution_id=375316653866487564, source_id=4617405884073871232, num_selected_g_fov=35, mean_obs_time_g_fov=2214.983747224728, time_duration_g_fov=935.00977, min_mag_g_fov=17.931557, max_mag_g_fov=18.669409, mean_mag_g_fov=18.303576, median_mag_g_fov=18.299463, range_mag_g_fov=0.73785233, trimmed_range_mag_g_fov=0.64042765, std_dev_mag_g_fov=0.25325578, skewness_mag_g_fov=0.017588899, kurtosis_mag_g_fov=-1.797536, mad_mag_g_fov=0.38241675, abbe_mag_g_fov=0.7652942, iqr_mag_g_fov=0.51978576, stetson_mag_g_fov=14.917244, std_dev_over_rms_err_mag_g_fov=4.1048636, outlier_median_g_fov=49.2365, num_selected_bp=32, mean_obs_time_bp=2214.8368581737377, time_duration_bp=935.00977, min_mag_bp=18.356804, max_mag_bp=19.26798, mean_mag_bp=18.627182, median_mag_bp=18.600204, range_mag_bp=0.911176, trimmed_range_mag_bp=0.4638848, std_dev_mag_bp=0.1833683, skewness_mag_bp=1.45152, kurtosis_mag_bp=3.3996491, mad_mag_bp=0.16367736, abbe_mag_bp=1.0174474,

In [28]:
vari.columns

['ra',
 'dec',
 'solution_id',
 'source_id',
 'num_selected_g_fov',
 'mean_obs_time_g_fov',
 'time_duration_g_fov',
 'min_mag_g_fov',
 'max_mag_g_fov',
 'mean_mag_g_fov',
 'median_mag_g_fov',
 'range_mag_g_fov',
 'trimmed_range_mag_g_fov',
 'std_dev_mag_g_fov',
 'skewness_mag_g_fov',
 'kurtosis_mag_g_fov',
 'mad_mag_g_fov',
 'abbe_mag_g_fov',
 'iqr_mag_g_fov',
 'stetson_mag_g_fov',
 'std_dev_over_rms_err_mag_g_fov',
 'outlier_median_g_fov',
 'num_selected_bp',
 'mean_obs_time_bp',
 'time_duration_bp',
 'min_mag_bp',
 'max_mag_bp',
 'mean_mag_bp',
 'median_mag_bp',
 'range_mag_bp',
 'trimmed_range_mag_bp',
 'std_dev_mag_bp',
 'skewness_mag_bp',
 'kurtosis_mag_bp',
 'mad_mag_bp',
 'abbe_mag_bp',
 'iqr_mag_bp',
 'stetson_mag_bp',
 'std_dev_over_rms_err_mag_bp',
 'outlier_median_bp',
 'num_selected_rp',
 'mean_obs_time_rp',
 'time_duration_rp',
 'min_mag_rp',
 'max_mag_rp',
 'mean_mag_rp',
 'median_mag_rp',
 'range_mag_rp',
 'trimmed_range_mag_rp',
 'std_dev_mag_rp',
 'skewness_mag_rp',
 '